### GPT-2 모델로 텍스트 생성하기 목차
* [Chapter 1 마스크 멀티 헤드 어텐션)](#chapter1)
* [Chapter 2 트랜스포머 디코더 모듈 만들기)](#chapter2)
* [Chapter 3 GPT-2 모델로 다양한 텍스트 생성하기)](#chapter3)


### Chapter 1 마스크 멀티 헤드 어텐션 <a class="anchor" id="chapter1"></a>
1. 트랜스포머 디코더(Transformer Decoder)는 원본 트랜스포머 모델에서 인코더가 이해한 내용을 바탕으로 결과를 생성하는 역활을 한다.
   - 영어 문장을 한글로 변역한다면 디코더는 인코더가 이해한 문장의 의미를 바탕으로 번역한다.
   - 각 단어를 예측할 때 지속적으로 이전 단어들과의 관계를 반영하기 때문에 사람이 쓴 것 처럼 높은 품질의 텍스트를 생성 할 수 있습니다.

2. 인코더-디코더 구조에서 디코더만 따로 분리해서 훈련할 수 있다.
    - 프롬프트가 입력되면 프롬프트 다음에 등장할 토큰을 생성하는 역활을 한다.
    - 훈련할 때의 타깃은 입력되는 텍스트 데이터의 다음 토큰이 된다.

3. 'Stay hungry, stay folish'라는 문장이 있다면, 'Stay'를 입력하면 'hungry'를 예측하는 식이다.
    - 이 때 디코더는 이전에 등장한 단어들만을 참고해서 다음 단어를 예측해야 한다.
    - 즉, 'stay'를 입력했을 때 'hungry'가 아니라 'folish'가 다시 예측되는 일이 없어야 한다.
    - 이를 위해 디코더는 마스크 멀티 헤드 어텐션(Masked Multi-Head Attention)이라는 기법을 사용한다.
    - 첫 번째 단어부터 토큰을 하나씩 이동하면서 다음 토큰을 예측하는 과정으로 쉽게 확장할 수 있다.
    - 디코더 모델이 'stay' 토큰을 사용해 'thund'를 예측했다. 하지만 정답은 'hungry'이다.
       - 두 단의 차이가 손실이 되고, 이 손실을 통해 모델의 가중치를 훈련한다.
       - 다음 등장할 토큰을 예측하기 위해 입력에 있는 'hungry' 토큰을 사용한다면 속임수를 쓰는 것과 같다.
       - 모델은 'stay' 토큰만을 바라봐야한다.
    - 입력 문장에서 하나의 토큰을 더 사용해 'stay hungry'가 입력되면 디코더 모델은 공백을 예측한다.
       - 정답 토큰은 ','이다.
       - 모델이 이전 단계에서 'thund'를 예측했지만 모델의 입력으로 사용하지 않았다.
       - 모델은 원복 텍스트에 있는 'stay hungry'를 입력으로 사용했다.
    - 마지막으로 하나의 토크을 더 사용해 'stay hungry,'를 입력으로 받았다.
       - 정답과 동이한 'stay'를 예측했다.       

        ![예측](image/05-01-predict2.png)   

4. 텍스트 생성 모델인 트랜스포머 디코더 모델은 입력 텍스트의 다음 토큰을 타깃으로 사용할 수 있다.
   - 이런한 훈련 방식을 자기지도 학습(self-supervised learning) 또는 코잘 언어 모델링(causal language modeling)이라고 한다.
   - 자기지도 학습 방식을 사용하면 훈련 데이터를 레이블링하는 수고를 들이지 않고도 많은 텍스트 데이터로부터 대규모 언어 모델을 훈련할 수 있다.
   - 여기서 핵심은 모델이 훈련 과정 중에 입력에 있는 다음 토큰을 훔쳐봐서는 않된다는 것이다.
   - 이를 위해 디코더는 마스크 멀티 헤드 어텐션(Masked Multi-Head Attention)이라는 기법을 사용한다.

5. 마스크 멀티 헤드 어텐션(Masked Multi-Head Attention)
    - 어텐션 점수를 계산할 때 현재 토큰에서 미래 토큰을 바라보지 못하도록 마스킹해 학습을 제한한다.
    - 5 * 5 크기의 어텐션 행렬에서 주 대각선 윗부분의 점수를 가린다.
        - 'hungry' 토큰을 처리할 때 다음에 나오는 ','와 'stay', 'foolish' 토큰에 대한 점수를 사용할 수 없다.
        - 어텐션 행렬은 셀프 어텐션 계산식의 쿼리와 키를 곱한 결과로, (n_tokens, n_tokens) 크기를 가진다.
        - 주 대간선은 왼쪽에서 오른쪽 아래까지 역슬래시(\) 모양으로 이어지는 대각선을 말한다.
        - MultiheadAttention 클래스의 attention_mask 매개변수에 마스킹 정보를 전달하기만 하면 자동으로 마스크 멀티해드 어텐션이 실행된다.
           - 이 마스킹을 종종 코잘 마스킹(causal masking)이라고 부른다.

        ![마스킹](image/05-01-masking2.png)   


In [5]:
import keras
from keras import layers
import keras

def make_causal_mask(seq_len):
    # keras.ops.arange() 함수를 사용하여 0부터 시퀸스 길이까지 채워진 텐서를 만든다.
    #   - 입력 값이 5인경우 [0, 1, 2, 3, 4] 형태의 텐서가 만들어진다.
    n_hori = keras.ops.arange(seq_len)
    
    # keras.ops.expand_dims() 함수를 사용하여 n_hori 텐서의 마지막 차원을 확장한다.
    #   - 입력 값이 [0, 1, 2, 3, 4] 인 경우 [[0], [1], [2], [3], [4]] 형태의 텐서가 만들어진다.
    #   - (seq_len, ) 형태의 텐서를 (seq_len, 1) 형태로 바꾼다.
    n_vert = keras.ops.expand_dims(n_hori, axis=-1)
    
    # n_vert >= n_hori 비교 연산을 수행하여 인덱스가 같거나 큰 위치에 True 값을 채운다.
    mask = n_vert >= n_hori
    return mask

   - expand_dims() 함수를 통해 2차원 텐서로 바뀐 텐서는 다음과 같다.
      - 입력 값이 [0, 1, 2, 3, 4] 인 경우 [[0], [1], [2], [3], [4]] 형태의 텐서가 만들어진다.

      ![마스킹3](image/05-01-masking3.png)   

   - 다시 비교 연산을 통해 다음과 같은 결과를 얻게된다.
      - mask = n_vert >= n_hori

      ![마스킹4](image/05-01-masking4.png)      

In [6]:
causal_mask = make_causal_mask(5)
print(causal_mask)

I0000 00:00:1759660503.383198    1426 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5555 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:08:00.0, compute capability: 8.6


tf.Tensor(
[[ True False False False False]
 [ True  True False False False]
 [ True  True  True False False]
 [ True  True  True  True False]
 [ True  True  True  True  True]], shape=(5, 5), dtype=bool)


In [7]:
padding_make = [1, 1, 1, 0, 0]
keras.ops.minimum(causal_mask, padding_make)

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0]], dtype=int32)>

- 결과 그림으로 표현

    ![마스킹5](image/05-01-masking5.png)      

In [8]:
def make_attention_mask(padding_mask):
    # padding_mask의 크기가 (2, 5) 라고 가정
    batch_size, seq_len = keras.ops.shape(padding_mask)
    # causal_mask의 크기는 (5, 5)
    causal_mask = make_causal_mask(seq_len)
    # 배치 차원을 추가해 (2, 5, 5) 크기로 만든다.
    causal_mask = keras.ops.broadcast_to(causal_mask, (batch_size, seq_len, seq_len))
    # 브로드캐스팅을 위해 padding_mask의 크기를 (2, 1, 5)로 바꾼다.
    padding_mask = keras.ops.expand_dims(padding_mask, axis=1)
    return keras.ops.minimum(causal_mask, padding_mask)

In [ ]:
# 첫 번재 코잘 마스킹 행렬은 첫 번재 핑딩 마스크를 따라서 두 번째 열 이후가 모두 마스킹
# 두 번째 코잘 마스킹 행렬은 두 번째 핑딩 마스크를 따라서 네 번째 열 이후가 모두 마스킹
make_attention_mask([[1, 1, 0, 0, 0], [1, 1, 1, 1, 0]])

<tf.Tensor: shape=(2, 5, 5), dtype=int32, numpy=
array([[[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 0, 0, 0]],

       [[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0]]], dtype=int32)>

### Chapter 2 트랜스포머 디코더 모듈 만들기 <a class="anchor" id="chapter2"></a>
1. 디코더 구조는 인코더-디코더 구조에서 사용될 때와 디코더 전용 모델에서 사용될 때의 구조가 다르다.
    - 인코더-디코더 구조에서는 디코더에 인코더의 출력을 받기 위한 장치가 있지만 디코더 전용 모델에서는 필요 없다.
    - 드롭아웃 다음에 있던 층 정규화가 스킵 연결 시작 부분으로 옮겨왔고, 멀티 페드 어텐션이 아니라 마스크드 헤드 어텐션이 구성된다.

    ![디코더](image/05-01-decoder.png)    

2. 케라스에서 입력 텐서에 대해 keras.ops 연산자를 사용하는 경우 사용자 정의층을 만들어야한다.
    - 향후에는 개선되겠지만, 현재는 make_attention_mask() 함수를 사용자 정의층으로 만들어야 한다.
    - AttentionMask 클래스는 keras.Layer 클래스를 상속하는 간단한 케라스 층으로, call() 메서드에서 make_attention_mask() 함수를 호출한다.

In [17]:
class AttentionMask(keras.Layer):
    def call(self, padding_mask):
        return make_attention_mask(padding_mask)

In [18]:
# norm_first 매개변수가 True일 경우, LayerNormalization 층이 스킵 연결 시작 부분에 위치한다.
# False일 경우 인코더와 동일하게 잔차 연결이 끝난 후 배치한다..
def transformer_decoder(x, padding_mask, dropout, activation='relu', norm_first=True):
    # 어텐션 마스크를 계산한다.
    attention_mask = AttentionMask()(padding_mask)
    
    # 스킵 연결을 준비한다.
    residul = x
    key_dim = hidden_dim // num_heads
    if norm_first:
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        
    # 멀티 헤드 어테션을 통과한다.
    #   - 매개변수에 패딩 마스크와 코잘 마스킹을 합친 어텐션 마스크를 전달한다.
    x = layers.MultiHeadAttention(num_heads, key_dim, dropout=dropout)(query=x, value=x, attention_mask=attention_mask)
    
    x = layers.Dropout(dropout)(x)
    
    # 스킵 연결을 수행한다.
    x = x + residul
    if not norm_first:
        x = layers.LayerNormalization(epsilon=1e-6)(x)
    
    # 스킵 연결을 준비한다.
    residul = x
    
    # 위치별 피드 포워드 네트워크
    if norm_first:
        x = layers.LayerNormalization(epsilon=1e-6)(x)
        
    # 두개의 밀집층으로 구성된다.
    #   - 첫 번째 밀집층의 유닛 개수는 은닉 차원의 4배이고, 활성화 함수는 ReLU를 사용한다.
    x = layers.Dense(hidden_dim * 4, activation=activation)(x)
    x = layers.Dense(hidden_dim)(x)
    x = layers.Dropout(dropout)(x)
    
    # 스킵 연결을 수행한다.
    x = x + residul
    if not norm_first:
        x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x
    

### Chapter 3 GPT-2 모델로 다양한 텍스트 생성하기 <a class="anchor" id="chapter3"></a>
1. GPT-1 모델은 2018년 오픈AI(OpenAI)에서 발표한 트랜스포머 디코더 기반의 언어 모델이다.
   - 12개의 트랜스포머 디코더 층을 쌓아 만들었으며, 1억 1700만 개의 매개변수를 가진다.
   - 위키피디아, 책, 뉴스 기사 등 다양한 도메인의 텍스트 데이터로 사전 훈련했다.
   - 사전 훈련된 GPT-1 모델은 문서 요약, 질문 답변, 번역 등 다양한 자연어 처리 과제를 잘 수행했다.
   - GPT-1 모델은 트랜스포머 디코더 구조를 사용해 텍스트 생성에 특화된 모델이다.
   - GPT-1 모델은 입력 텍스트의 다음 토큰을 예측하는 자기지도 학습 방식을 사용해 사전 훈련했다.
   - GPT-1 모델은 프롬프트(prompt)를 입력받아 프롬프트 다음에 등장할 토큰을 생성하는 역활을 한다.
      - 프롬프트는 '시작' 또는 '단서'라는 뜻으로, 텍스트 생성 모델에 주어지는 입력 텍스트를 의미한다.
      - 예를 들어, 'The capital of France is'라는 프롬프트를 입력하면 'Paris'라는 토큰을 생성한다.
   - GPT-1 모델은 프롬프트 다음에 등장할 토큰을 하나씩 생성하는 과정을 반복해 텍스트를 생성한다.
      - 예를 들어, 'The capital of France is'라는 프롬프트를 입력하면 'Paris is a beautiful city'라는 문장을 생성할 수 있다.

2. GPT-2 모델은 2019년 오픈AI(OpenAI)에서 발표한 트랜스포머 디코더 기반의 언어 모델이다.
   - 1.5억 개의 매개변수를 가진 GPT-1 모델과 달리, GPT-2 모델은 15억 개의 매개변수를 가진다.
   - 2017년 12월 이전에 레딧 사이트에 적어도 세 번의 추천을 받은 링크를 대상으로 데이터를 수집했다.
   - 수집된 데이터는 약 40GB 크기의 텍스트 데이터로 WebText라고 부르며, 책, 위키피디아, 뉴스 기사 등 다양한 도메인의 텍스트 데이터를 포함한다.
   - GPT-1과 다른 점은 층 정규화를 스킵 연결 시작 부분으로 옮겨왔고, 드롭아웃 비율을 0.1에서 0.3으로 늘렸다는 점이다.
   - RoBERTa 모델과 비교해 임베딩 직후에 노여 있던 층 정규화가 디코더 뒤쪽으로 이동했다.
   - GTP-2는 분류기 대신 토큰 임베딩층을 뒤집어 어휘사전 크기에 해당하는 출력을 만드는 리버스 임베딩을 사용한다.

      ![GPT2](image/05-01-gpt2.png)    

In [10]:
# gpt2_base_en 모델의 하이퍼파라미터
vocab_size = 50257 # GPT-2 토크나이저의 어휘 사전 크기
num_layers = 12 # 디코더 층 개수
num_heads = 12 # 멀티 헤드 어텐션의 헤드 개수
hidden_dim = 768 # 은닉층 차원
dropout = 0.1 # 드롭아웃 비율
activation = 'gelu' # 활성화 함수
max_len = 1024 # 최대 시퀀스 길이

token_ids = keras.Input(shape=(None,), ) # 정수 인덱스 시퀀스를 입력 받는다.
padding_mask = keras.Input(shape=(None,), ) # 패딩 마스크 

In [13]:
import keras
from keras import layers
import keras_nlp
from keras_nlp.layers import ReversibleEmbedding

# 임베딩 층 정의
#   - 모델의 마지막 부분에서 768 차원의 출력을 어휘사전 크기에 해당하는 50,257 차원으로 다시 매핑하는 역할을 한다.
#   - 이 차원을 따라 가장 큰 값을 갖는 위치가 다음 토큰의 인덱스가 된다.
token_embedding_layer = ReversibleEmbedding(vocab_size, hidden_dim)

token_embedding = embedding_layer(token_ids) # 토큰 임베딩
pos_embedding = keras_nlp.layers.PositionEmbedding(max_len)(token_embedding) # 위치 임베딩

In [20]:
x = token_embedding + pos_embedding # 토큰 임베딩과 위치 임베딩을 더한다.
x = layers.Dropout(dropout)(x) # 드롭아웃 적용
for _ in range(num_layers):
    x = transformer_decoder(x, padding_mask, dropout, activation)
    
x = layers.LayerNormalization(epsilon=1e-6)(x) # 층 정규화
outputs = token_embedding_layer(x, reverse=True) # 출력 임베딩
model = keras.Model(inputs=(token_ids, padding_mask), outputs=outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reversible_embeddi… │ (None, None, 768) │ 38,597,376 │ input_layer[0][0] │
│ (ReversibleEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding… │ (None, None, 768) │    786,432 │ reversible_embed… │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_27 (Add)        │ (None, None, 768) │          0 │ reversible_embed… │
│                     │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_39          │ (None, None, 768) │          0 │ add_27[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask_12   │ (None, None,      │          0 │ input_layer_1[0]… │
│ (AttentionMask)     │ None)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 768) │      1,536 │ dropout_39[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, None, 768) │  2,362,368 │ attention_mask_1… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_41          │ (None, None, 768) │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_28 (Add)        │ (None, None, 768) │          0 │ dropout_41[0][0], │
│                     │                   │            │ dropout_39[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, None, 768) │      1,536 │ add_28[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, None,      │  2,362,368 │ layer_normalizat… │
│                     │ 3072)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, None, 768) │  2,360,064 │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_42          │ (None, None, 768) │          0 │ dense_25[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_29 (Add)        │ (None, None, 768) │          0 │ dropout_42[0][0], │
│                     │                   │            │ add_28[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask_13   │ (None, None,      │          0 │ input_layer_1[0]

 Total params: 163,037,184 (621.94 MB)

 Trainable params: 163,037,184 (621.94 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# KerasNLP에 있는 사전 훈련된 GPT-2 모델을 사용하여 텍스트를 생성한다.
gpt2 = keras_nlp.models.GPT2CausalLM.from_preset('gpt2_base_en')
gpt2.summary()

100%|██████████| 618/618 [00:00<00:00, 1.22MB/s]


100%|██████████| 0.99M/0.99M [00:01<00:00, 853kB/s]


100%|██████████| 446k/446k [00:00<00:00, 458kB/s]


Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                                │                       Vocab size: 50,257 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │     124,439,808 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      38,597,376 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

1. 'stay hungry, stay'로 시작하는 문장만들기
    - 디코더 모델은 입력 텍스트의 다음 단어를 예측하도록 훈련된다.
    - 훈련된 모델을 사용할 때 다음 단어를 예측하려면 최초의 입력 텍스트가 필요한데 이를 종종 프롬프트(prompt)라고 부른다.

2. 훈련된 모델로 새로운 텍스트를 생성할 때는 정답 텍스트가 없기 때문에 초기 프로프트에서 다음 토큰을 예측하고, 이를 다시 프롬프트에 이어 붙인다.
   - 이런 방식을 종종 자기회귀 모델이라고 부른다.

     ![자기회귀 모델](image/05-01-gpt3.png)  

   - GPT-2 모델은 최대 1,024개의 토큰을 입력으로 받을 수 있기 때문에, 생성되는 텍스트도 최대 1,024개의 토큰까지만 만들 수 있다.

In [25]:
gpt2.generate("stay hungry, stay", max_length=6)

I0000 00:00:1759672558.232134    1426 service.cc:152] XLA service 0xc073b70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759672558.232286    1426 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-10-05 22:55:58.317559: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1759672558.945531    1426 cuda_dnn.cc:529] Loaded cuDNN version 91002
2025-10-05 22:56:00.524285: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_238', 484 bytes spill stores, 484 bytes spill loads

2025-10-05 22:56:01.132337: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_259', 

'stay hungry, stay thirsty'

In [26]:
gpt2.generate("stay hungry, stay", max_length=20)

2025-10-05 22:56:34.323882: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_238', 4 bytes spill stores, 4 bytes spill loads

2025-10-05 22:56:34.517727: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_238_0', 8 bytes spill stores, 8 bytes spill loads

2025-10-05 22:56:34.589968: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_238', 640 bytes spill stores, 640 bytes spill loads

2025-10-05 22:56:34.651073: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_238', 300 bytes spill stores, 300 bytes spill loads

2025-10-05 22:56:35.097895: I external/loc

'stay hungry, stay thirsty and stay strong\n\nthe world is your oyster\n\nthe'

In [ ]:
# gpt2 전처리 수행 확인
inputs, target, mask = gpt2.preprocessor("stay hungry, stay", sequence_length=10)
inputs, target, mask

# 첫 번째 stay(31712)와 두 번째 stay(2652)의 토큰아이디가 다르다.
#   - GPT가 사용하는 바이트 수준의 BPE 토크나이저는 토큰의 앞의 공백을 토큰에 포함시킨다.

({'token_ids': <tf.Tensor: shape=(10,), dtype=int32, numpy=
  array([50256, 31712, 14720,    11,  2652, 50256,     0,     0,     0,
             0], dtype=int32)>,
  'padding_mask': <tf.Tensor: shape=(10,), dtype=bool, numpy=
  array([ True,  True,  True,  True,  True,  True, False, False, False,
         False])>},
 <tf.Tensor: shape=(10,), dtype=int32, numpy=
 array([31712, 14720,    11,  2652, 50256,     0,     0,     0,     0,
            0], dtype=int32)>,
 <tf.Tensor: shape=(10,), dtype=bool, numpy=
 array([ True,  True,  True,  True,  True, False, False, False, False,
        False])>)

In [ ]:
gpt2_tokenizer = gpt2.preprocessor.tokenizer
for ids in target:
    print(gpt2_tokenizer.id_to_token(ids), end=' ')
# 첫 번째 토큰 50256 다음 토큰부터 타깃으로 시작되기 때문에 앞에 50256이 출력되지 않는다.

stay Ġhungry , Ġstay <|endoftext|> ! ! ! ! ! 

In [ ]:
# generate_preprocess() 메서드는 배치 입력을 기대하기 때문에 하나의 문자열도 리스트로 감싸서 전달해야한다.
#   - 반환된 값은 토큰 아이디와 패딩 마스크를 담은 딕셔너리이다.
#   - 텍스트를 생성을 이어가기 위해 시작 토큰이나 종료 토큰이 없다.
inputs = gpt2.preprocessor.generate_preprocess(['stay hungry, stay'], sequence_length=10)
inputs

{'token_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
 array([[50256, 31712, 14720,    11,  2652,     0,     0,     0,     0,
             0]], dtype=int32)>,
 'padding_mask': <tf.Tensor: shape=(1, 10), dtype=bool, numpy=
 array([[ True,  True,  True,  True,  True, False, False, False, False,
         False]])>}

In [ ]:
# generate_preprocess() 메서드가 반환한 값을 generate_function() 메서드에 전달하여 텍스트를 생성한다.
outputs = gpt2.generate_function(inputs)
outputs

2025-10-05 23:08:09.996256: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_238', 556 bytes spill stores, 556 bytes spill loads

2025-10-05 23:08:10.623436: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_259', 556 bytes spill stores, 556 bytes spill loads



{'token_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
 array([[50256, 31712, 14720,    11,  2652, 14720,    11,  2652, 14720,
             0]], dtype=int32)>,
 'padding_mask': <tf.Tensor: shape=(1, 10), dtype=bool, numpy=
 array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True]])>}

In [33]:
gpt2.preprocessor.generate_postprocess(outputs)

['stay hungry, stay hungry, stay hungry!']